In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, LSTM, GRU, Flatten, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = '/kaggle/input/vaildated/validated.csv'
data = pd.read_csv(file_path)

# Basic data preprocessing
# Assuming the target column is the last one (adjust if necessary)
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Handle categorical target if necessary
if y.dtype == 'object':
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
y = to_categorical(y)  # Convert to one-hot encoding for multi-class classification

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for CNN and RNN inputs (1D convolution and RNNs require 3D input: samples, timesteps, features)
X_train_cnn_rnn = np.expand_dims(X_train, axis=2)
X_test_cnn_rnn = np.expand_dims(X_test, axis=2)

# Build the combined FNN, CNN, and RNN (LSTM/GRU) model
input_layer_fnn = Input(shape=(X_train.shape[1],))

# FNN branch
fnn = Dense(128, activation='relu')(input_layer_fnn)
fnn = Dropout(0.3)(fnn)
fnn = Dense(64, activation='relu')(fnn)
fnn = Dropout(0.3)(fnn)

# CNN branch
input_layer_cnn = Input(shape=(X_train_cnn_rnn.shape[1], X_train_cnn_rnn.shape[2]))
cnn = Conv1D(64, kernel_size=3, activation='relu')(input_layer_cnn)
cnn = Flatten()(cnn)

# RNN branch (using LSTM and GRU)
rnn = LSTM(64, return_sequences=True)(input_layer_cnn)
rnn = GRU(32)(rnn)

# Combine branches
combined = concatenate([fnn, cnn, rnn])
output_layer = Dense(y.shape[1], activation='softmax')(combined)  # Use 'softmax' for multi-class classification

# Define the model
model = Model(inputs=[input_layer_fnn, input_layer_cnn], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    [X_train, X_train_cnn_rnn], y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
eval_results = model.evaluate([X_test, X_test_cnn_rnn], y_test, verbose=0)
print(f"Test Accuracy: {eval_results[1] * 100:.2f}%")

# Save the model
model.save('best_combined_model_with_rnn_multiclass.h5')

# Plot accuracy and loss curves (optional)
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
